# EDA precipitación de Azure Table Storage


### Librerías


In [6]:
from sklearn.experimental import enable_iterative_imputer
import pandas as pd
import os
import plotly.express as px
import dash
from data_service import set_table_service, get_dataframe_from_table_storage_table
from dotenv import load_dotenv
from dash import dcc, html
from dash.dependencies import Input, Output
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [7]:
load_dotenv()

True

In [8]:
connection_string = os.getenv("AZ_CONNECTION_STRING")

### Creando DataFrame con Data de Az Table Storage


In [9]:
table_name_precipitacion = "PRECIPITACIONCurated"
filterQuery = "PartitionKey ne 'random'"

In [10]:
table_service_precipitacion = set_table_service(
    connection_string, table_name_precipitacion
)
df_precipitacion = get_dataframe_from_table_storage_table(
    table_service_precipitacion, filterQuery
)
print(f"Shape: {df_precipitacion.shape}")
print(f"dtypes: {df_precipitacion.dtypes}")
df_precipitacion.head()

Shape: (261362, 13)
dtypes: TimeStamp                int64
codigoestacion          object
codigosensor            object
date                    object
departamento            object
descripcionsensor       object
latitud                 object
longitud                object
municipio               object
nombreestacion          object
precipitacion_total    float64
unidadmedida            object
zonahidrografica        object
dtype: object


,TimeStamp,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,20231026,0042077020,0240,2021-07-14T00:00:00.000,VAUPÉS,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,20231026,0012015110,0240,2021-12-12T00:00:00.000,ANTIOQUIA,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL
2,20231026,3526500201,0240,2023-01-17T00:00:00.000,VICHADA,Precipitacion,5.48088889,-70.42130556,LA PRIMAVERA,LA PRIMAVERA,0.0,mm,ORINOCO
3,20231026,0024025030,0240,2021-11-15T00:00:00.000,BOYACÁ,Precipitacion,5.966388889,-73.16389139,PAIPA,LA SIERRA - AUT,5.0,mm,SOGAMOSO
4,20231026,0035025110,0240,2023-01-02T00:00:00.000,META,Precipitacion,4.057361111,-73.46791667,VILLAVICENCIO,LA LIBERTAD - AUT,0.5,mm,META


In [11]:
df_precipitacion = df_precipitacion.drop(
    ["TimeStamp"], axis=1
)
df_precipitacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261362 entries, 0 to 261361
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   codigoestacion       261362 non-null  object 
 1   codigosensor         261362 non-null  object 
 2   date                 261362 non-null  object 
 3   departamento         261362 non-null  object 
 4   descripcionsensor    261362 non-null  object 
 5   latitud              261362 non-null  object 
 6   longitud             261362 non-null  object 
 7   municipio            261362 non-null  object 
 8   nombreestacion       261362 non-null  object 
 9   precipitacion_total  261362 non-null  float64
 10  unidadmedida         261362 non-null  object 
 11  zonahidrografica     261362 non-null  object 
dtypes: float64(1), object(11)
memory usage: 23.9+ MB


In [12]:
df_precipitacion_sin_duplicados = df_precipitacion.drop_duplicates(subset=['codigoestacion','date','municipio','departamento'])
df_precipitacion_sin_duplicados_fecha = df_precipitacion_sin_duplicados.copy()

df_precipitacion_sin_duplicados_fecha['date'] = pd.to_datetime(df_precipitacion_sin_duplicados_fecha['date'], errors='coerce')
df_precipitacion_sin_duplicados_fecha['date'] = df_precipitacion_sin_duplicados_fecha['date'].dt.strftime('%Y-%m-%d')
df_precipitacion_sin_duplicados_fecha['date'] = pd.to_datetime(df_precipitacion_sin_duplicados_fecha['date'])

df_precipitacion_sin_duplicados_fecha.info()
df_precipitacion_sin_duplicados_fecha.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 257538 entries, 0 to 261361
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   codigoestacion       257538 non-null  object        
 1   codigosensor         257538 non-null  object        
 2   date                 257538 non-null  datetime64[ns]
 3   departamento         257538 non-null  object        
 4   descripcionsensor    257538 non-null  object        
 5   latitud              257538 non-null  object        
 6   longitud             257538 non-null  object        
 7   municipio            257538 non-null  object        
 8   nombreestacion       257538 non-null  object        
 9   precipitacion_total  257538 non-null  float64       
 10  unidadmedida         257538 non-null  object        
 11  zonahidrografica     257538 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 25.5+ MB


,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,0042077020,0240,2021-07-14,VAUPÉS,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,0012015110,0240,2021-12-12,ANTIOQUIA,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL


Revisión de datos retirados


In [13]:
merged_df = pd.merge(df_precipitacion, df_precipitacion_sin_duplicados, how='outer', indicator=True)
removed_data = merged_df[merged_df['_merge'] == 'left_only']
print("Datos Retirados:")
pd.DataFrame(removed_data)
removed_data.info()
removed_data.head(2)

Datos Retirados:
<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 24547 to 261019
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   codigoestacion       235 non-null    object  
 1   codigosensor         235 non-null    object  
 2   date                 235 non-null    object  
 3   departamento         235 non-null    object  
 4   descripcionsensor    235 non-null    object  
 5   latitud              235 non-null    object  
 6   longitud             235 non-null    object  
 7   municipio            235 non-null    object  
 8   nombreestacion       235 non-null    object  
 9   precipitacion_total  235 non-null    float64 
 10  unidadmedida         235 non-null    object  
 11  zonahidrografica     235 non-null    object  
 12  _merge               235 non-null    category
dtypes: category(1), float64(1), object(11)
memory usage: 24.2+ KB


,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica,_merge
24547,0016015501,0257,2023-09-18T00:00:00.000,NORTE DE SANTANDER,GPRS - PRECIPITACIÓN,7.93028,-72.50917,CÚCUTA,AEROPUERTO CAMILO DAZA,0.0,mm,CATATUMBO,left_only
26046,3502500135,0257,2023-09-27T00:00:00.000,CUNDINAMARCA,GPRS - PRECIPITACIÓN,4.22553,-73.81481,GUAYABETAL,GUAYABETAL POLLO OLIMPICO,0.0,mm,META,left_only


In [14]:
conteo_por_departamento = df_precipitacion_sin_duplicados_fecha["departamento"].value_counts()
conteo_por_departamento = conteo_por_departamento.sort_index()
print(conteo_por_departamento)

departamento
AMAZONAS                                                      750
ANTIOQUIA                                                   17916
ARAUCA                                                       1800
ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA      497
ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA        49
ATLANTICO                                                    1841
ATLÁNTICO                                                    2876
BOGOTA                                                       2522
BOGOTA D.C.                                                   816
BOGOTÁ                                                         38
BOLIVAR                                                      6613
BOLÍVAR                                                       568
BOYACÁ                                                      28194
CALDAS                                                      14302
CAQUETA                                                      23

In [15]:
df_estandarizado = df_precipitacion_sin_duplicados_fecha.copy()

# Estandarizar el nombre del departamento
df_estandarizado["departamento"] = (
    df_estandarizado["departamento"]
    .str.lower()  # Convertir a minúsculas
    .str.replace("d.c.", "", regex=False)  # Retirar "D.C."
    .str.replace("[^\w\s]", "", regex=True)  # Retirar comas, puntos
    .str.replace("[áäâà]", "a", regex=True)  # Reemplazar tildes
    .str.replace("[éëêè]", "e", regex=True)
    .str.replace("[ñ]", "n", regex=True)
    .str.replace("[íïîì]", "i", regex=True)
    .str.replace("[óöôò]", "o", regex=True)
    .str.replace("[úüûù]", "u", regex=True)
    .str.strip()  # Retirar espacios al principio y al final
)
df_estandarizado.loc[
    df_estandarizado["departamento"].str.contains("san andres providencia", case=False),
    "departamento",
] = "san andres providencia"

df_estandarizado["date"] = pd.to_datetime(df_estandarizado["date"])
# Verificar el DataFrame estandarizado
df_estandarizado.head(2)

,codigoestacion,codigosensor,date,departamento,descripcionsensor,latitud,longitud,municipio,nombreestacion,precipitacion_total,unidadmedida,zonahidrografica
0,0042077020,0240,2021-07-14,vaupes,Precipitacion,1.26,-70.239,MITÚ,MITU,10.0,mm,VAUPES
1,0012015110,0240,2021-12-12,antioquia,Precipitacion,7.671138889,-76.69405556,CHIGORODÓ,CHIGORODO - AUT,0.1,mm,CARIBE - LITORAL


In [16]:
conteo_por_departamento = df_estandarizado["departamento"].value_counts()
conteo_por_departamento = conteo_por_departamento.sort_index()
print(conteo_por_departamento)

departamento
amazonas                    750
antioquia                 17916
arauca                     1800
atlantico                  4717
bogota                     3376
bolivar                    7181
boyaca                    28194
caldas                    14302
caqueta                    2404
casanare                   6683
cauca                      8500
cesar                      4315
choco                      6540
cordoba                    9010
cundinamarca              26545
guainia                     633
guaviare                    213
huila                     14266
la guajira                 4038
magdalena                  7136
meta                       6071
narino                    11769
norte de santander        10337
putumayo                   1754
quindio                    5281
risaralda                  6638
san andres providencia     2029
santander                 15178
sucre                      4067
tolima                    13317
valle del cauca           1

In [17]:
df_agrupado = (
    df_estandarizado.groupby(["date", "departamento"])
    .agg(
        {
            "precipitacion_total": "sum",
            "unidadmedida": "first",  # Tomar el primer valor de 'unidadmedida' en cada grupo
        }
    )
    .reset_index()
)

# Verificar el DataFrame resultante
df_agrupado.info()
df_agrupado.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27885 entries, 0 to 27884
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 27885 non-null  datetime64[ns]
 1   departamento         27885 non-null  object        
 2   precipitacion_total  27885 non-null  float64       
 3   unidadmedida         27885 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 871.5+ KB


,date,departamento,precipitacion_total,unidadmedida
0,2021-01-01,amazonas,49.110,mm
1,2021-01-01,antioquia,227.376,mm
2,2021-01-01,arauca,7.800,mm
3,2021-01-01,atlantico,11.230,mm
4,2021-01-01,bogota,0.110,mm


In [18]:
# Gráfica 1
df_diario = (
    df_agrupado.groupby(["date", "departamento"])
    .agg({"precipitacion_total": "sum"})
    .reset_index()
)

lista_departamentos_grafica1 = df_diario["departamento"].unique()

app1 = dash.Dash(__name__)

app1.layout = html.Div(
    [
        dcc.Dropdown(
            id="departamento-dropdown-grafica1",
            options=[
                {"label": departamento, "value": departamento}
                for departamento in lista_departamentos_grafica1
            ],
            value=lista_departamentos_grafica1[0],
            multi=False,
        ),
        dcc.Graph(id="graph1"),
    ]
)


@app1.callback(
    Output("graph1", "figure"), [Input("departamento-dropdown-grafica1", "value")]
)
def update_graph1(selected_departamento):
    df_selected_departamento = df_diario[
        df_diario["departamento"] == selected_departamento
    ]

    df_pivot = df_selected_departamento.pivot(
        index="date", columns="departamento", values="precipitacion_total"
    ).reset_index()

    df_pivot["year"] = df_pivot["date"].dt.year
    df_pivot["month"] = df_pivot["date"].dt.month

    fig1 = px.line(
        df_pivot,
        x="date",
        y=selected_departamento,
        color="year",
        labels={selected_departamento: "Precipitación Total"},
        title=f"Precipitación Total Diaria en {selected_departamento}",
    )

    return fig1



# Ejecutar la aplicación
if __name__ == "__main__":
    app1.run_server(debug=True, port=8050)

Identificamos vacios en la continuidad en los registros de precipitación de los departamentos, encontrando que vaupes, guaviare, guainia y amazonas son los más afectados.

In [19]:
# Completar el DataFrame con fechas continuas
df_continuo = df_agrupado.copy()

fechas_continuas = pd.date_range(
    start=df_continuo["date"].min(), end=df_continuo["date"].max(), freq="D"
)

df_continuo = pd.DataFrame({"date": fechas_continuas}).drop_duplicates()

# Combinar el DataFrame continuo con el DataFrame original
df_continuo = pd.merge(df_continuo, df_agrupado, on="date", how="left")

# Llenar valores NaN con 0 en la columna 'precipitacion_total'
df_continuo["precipitacion_total"] = df_continuo["precipitacion_total"].fillna(0)

# Pivote del DataFrame
df_pivot_2 = df_continuo.pivot_table(
    index=["date", "unidadmedida"],
    columns="departamento",
    values="precipitacion_total",
    aggfunc="sum",
).reset_index()
df_pivot_2 = df_pivot_2.rename_axis(index={'departamento': 'date'})
df_pivot_2.set_index("date", inplace=True)
fechas_continuas = pd.date_range(
    start=df_pivot_2.index.min(), end=df_pivot_2.index.max()
)
df_pivot_2 = df_pivot_2.reindex(fechas_continuas)
df_pivot_2.rename(columns={'index': 'date'}, inplace=True)
# Imprime el DataFrame resultante
df_pivot_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1003 entries, 2021-01-01 to 2023-09-30
Freq: D
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unidadmedida            959 non-null    object 
 1   amazonas                555 non-null    float64
 2   antioquia               940 non-null    float64
 3   arauca                  854 non-null    float64
 4   atlantico               936 non-null    float64
 5   bogota                  894 non-null    float64
 6   bolivar                 937 non-null    float64
 7   boyaca                  942 non-null    float64
 8   caldas                  886 non-null    float64
 9   caqueta                 889 non-null    float64
 10  casanare                936 non-null    float64
 11  cauca                   914 non-null    float64
 12  cesar                   927 non-null    float64
 13  choco                   915 non-null    float64
 14  cordoba       

In [20]:
df_continuo = df_agrupado.copy()

fechas_continuas = pd.date_range(
    start=df_continuo["date"].min(), end=df_continuo["date"].max(), freq="D"
)

df_continuo = pd.DataFrame({"date": fechas_continuas})

# Combinar el DataFrame continuo con el DataFrame original
df_continuo = pd.merge(df_continuo, df_agrupado, on="date", how="left")

# Pivote del DataFrame
df_pivot_2 = df_continuo.pivot_table(
    index=["date", "unidadmedida"],
    columns="departamento",
    values="precipitacion_total",
    aggfunc="sum",
).reset_index()

# Rename the index and set it to 'date'
df_pivot_2.index.name = 'date'
df_pivot_2.set_index("date", inplace=True)

# Reindex to fill missing dates
fechas_continuas = pd.date_range(
    start=df_pivot_2.index.min(), end=df_pivot_2.index.max(), freq='D'
)
df_pivot_2 = df_pivot_2.reindex(fechas_continuas)

# Reset the index to make 'date' a regular column
df_pivot_2.reset_index(inplace=True)

# Rename the columns
df_pivot_2.columns.name = None

df_pivot_2 = df_pivot_2.rename(columns={'index': 'date'})
df_pivot_2['unidadmedida']='mm'
# Imprime el DataFrame resultante
df_pivot_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    1003 non-null   datetime64[ns]
 1   unidadmedida            1003 non-null   object        
 2   amazonas                555 non-null    float64       
 3   antioquia               940 non-null    float64       
 4   arauca                  854 non-null    float64       
 5   atlantico               936 non-null    float64       
 6   bogota                  894 non-null    float64       
 7   bolivar                 937 non-null    float64       
 8   boyaca                  942 non-null    float64       
 9   caldas                  886 non-null    float64       
 10  caqueta                 889 non-null    float64       
 11  casanare                936 non-null    float64       
 12  cauca                   914 non-null    float64 

Generamos un nuevo dataframe con continuidad en todas las observaciones.

IterativeImputer

In [21]:
df_precipitacion_completo_iterativeimputer = df_pivot_2.copy()

columns_to_impute = [col for col in df_precipitacion_completo_iterativeimputer if col != "date" and col != "unidadmedida"]

# Crear un imputador IterativeImputer
imputer = IterativeImputer(random_state=0)

# Aplicar la imputación solo a las columnas numéricas
df_precipitacion_completo_iterativeimputer[columns_to_impute] = imputer.fit_transform(
    df_precipitacion_completo_iterativeimputer[columns_to_impute]
)

df_precipitacion_completo_iterativeimputer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    1003 non-null   datetime64[ns]
 1   unidadmedida            1003 non-null   object        
 2   amazonas                1003 non-null   float64       
 3   antioquia               1003 non-null   float64       
 4   arauca                  1003 non-null   float64       
 5   atlantico               1003 non-null   float64       
 6   bogota                  1003 non-null   float64       
 7   bolivar                 1003 non-null   float64       
 8   boyaca                  1003 non-null   float64       
 9   caldas                  1003 non-null   float64       
 10  caqueta                 1003 non-null   float64       
 11  casanare                1003 non-null   float64       
 12  cauca                   1003 non-null   float64 

/home/dalopeza/.local/share/virtualenvs/Proyecto-Integrador-Semestre1-VZ_FhVF5/lib/python3.11/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning:

[IterativeImputer] Early stopping criterion not reached.



In [22]:
lista_departamentos_grafica2 = df_precipitacion_completo_iterativeimputer.columns[2:]

app2 = dash.Dash(__name__)

app2.layout = html.Div(
    [
        dcc.Dropdown(
            id="departamento-dropdown-grafica2",
            options=[
                {"label": departamento, "value": departamento}
                for departamento in lista_departamentos_grafica2
            ],
            value=lista_departamentos_grafica2[0],
            multi=False,
        ),
        dcc.Graph(id="graph2"),
    ]
)


@app2.callback(
    Output("graph2", "figure"), [Input("departamento-dropdown-grafica2", "value")]
)
def update_graph2(selected_departamento):
    df_selected_departamento = df_precipitacion_completo_iterativeimputer[
        ["date", "unidadmedida", selected_departamento]
    ].copy()

    fig2 = px.line(
        df_selected_departamento,
        x="date",
        y=selected_departamento,
        labels={selected_departamento: "Precipitación Total"},
        title=f"Precipitación Total Diaria en {selected_departamento}",
    )

    return fig2


# Ejecutar la aplicación
if __name__ == "__main__":
    app2.run_server(debug=True, port=8051)

KNNImputer

In [23]:
df_precipitacion_completo_knn = df_pivot_2.copy()

# Seleccionar solo las columnas numéricas para la imputación
columns_to_impute = df_precipitacion_completo_knn.columns[2:]

# Crear un imputador KNNImputer con el número deseado de vecinos (k)
knn_imputer = KNNImputer(n_neighbors=5)

# Aplicar la imputación solo a las columnas numéricas
df_precipitacion_completo_knn[columns_to_impute] = knn_imputer.fit_transform(
    df_precipitacion_completo_knn[columns_to_impute]
)

# Imprime el DataFrame resultante
df_precipitacion_completo_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    1003 non-null   datetime64[ns]
 1   unidadmedida            1003 non-null   object        
 2   amazonas                1003 non-null   float64       
 3   antioquia               1003 non-null   float64       
 4   arauca                  1003 non-null   float64       
 5   atlantico               1003 non-null   float64       
 6   bogota                  1003 non-null   float64       
 7   bolivar                 1003 non-null   float64       
 8   boyaca                  1003 non-null   float64       
 9   caldas                  1003 non-null   float64       
 10  caqueta                 1003 non-null   float64       
 11  casanare                1003 non-null   float64       
 12  cauca                   1003 non-null   float64 

In [24]:
lista_departamentos_grafica2 = df_precipitacion_completo_knn.columns[2:]

app3 = dash.Dash(__name__)

app3.layout = html.Div(
    [
        dcc.Dropdown(
            id="departamento-dropdown-grafica3",
            options=[
                {"label": departamento, "value": departamento}
                for departamento in lista_departamentos_grafica2
            ],
            value=lista_departamentos_grafica2[0],
            multi=False,
        ),
        dcc.Graph(id="graph3"),
    ]
)


@app3.callback(
    Output("graph3", "figure"), [Input("departamento-dropdown-grafica3", "value")]
)
def update_graph3(selected_departamento):
    df_selected_departamento = df_precipitacion_completo_knn[
        ["date", "unidadmedida", selected_departamento]
    ].copy()

    fig3 = px.line(
        df_selected_departamento,
        x="date",
        y=selected_departamento,
        labels={selected_departamento: "Precipitación Total"},
        title=f"Precipitación Total Diaria en {selected_departamento}",
    )

    return fig3


# Ejecutar la aplicación
if __name__ == "__main__":
    app3.run_server(debug=True, port=8052)

Moving average

In [25]:
def impute_moving_average(df, window_size=7):
    df_imputed = df.copy()

    for column in df.columns[2:]:
        series = df[column]

        # Fill missing values with a method of your choice (e.g., forward fill, backward fill, or interpolation)
        series_filled = series.ffill()

        # Calculate the moving average
        moving_avg = series_filled.rolling(window=window_size, min_periods=1, center=True).mean()

        # Impute missing values using the moving average
        imputed_values = moving_avg
        df_imputed[column] = df_imputed[column].combine_first(pd.Series(imputed_values, index=df_imputed.index))

    return df_imputed

# Impute missing values using moving average with a window size of 7 days
df_precipitacion_completo_ma = impute_moving_average(df_pivot_2, window_size=7)

# Print the DataFrame
print(df_precipitacion_completo_ma.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    1003 non-null   datetime64[ns]
 1   unidadmedida            1003 non-null   object        
 2   amazonas                1003 non-null   float64       
 3   antioquia               1003 non-null   float64       
 4   arauca                  1003 non-null   float64       
 5   atlantico               1003 non-null   float64       
 6   bogota                  1003 non-null   float64       
 7   bolivar                 1003 non-null   float64       
 8   boyaca                  1003 non-null   float64       
 9   caldas                  1003 non-null   float64       
 10  caqueta                 1003 non-null   float64       
 11  casanare                1003 non-null   float64       
 12  cauca                   1003 non-null   float64 

In [26]:
lista_departamentos_grafica3 = df_precipitacion_completo_ma.columns[2:]

app4 = dash.Dash(__name__)

app4.layout = html.Div(
    [
        dcc.Dropdown(
            id="departamento-dropdown-grafica3",
            options=[
                {"label": departamento, "value": departamento}
                for departamento in lista_departamentos_grafica3
            ],
            value=lista_departamentos_grafica3[0],
            multi=False,
        ),
        dcc.Graph(id="graph3"),
    ]
)


@app4.callback(
    Output("graph3", "figure"), [Input("departamento-dropdown-grafica3", "value")]
)
def update_graph3(selected_departamento):
    df_selected_departamento = df_precipitacion_completo_ma[
        ["date", "unidadmedida", selected_departamento]
    ].copy()

    fig4 = px.line(
        df_selected_departamento,
        x="date",
        y=selected_departamento,
        labels={selected_departamento: "Precipitación Total"},
        title=f"Precipitación Total Diaria en {selected_departamento}",
    )

    return fig4


# Ejecutar la aplicación
if __name__ == "__main__":
    app4.run_server(debug=True, port=8053)

In [36]:
df_represas = pd.read_csv("./processed_tables/represas.csv")
departamentos = df_represas["represas_Departamento"].tolist()
len(departamentos)

10

In [41]:
df_precipitacion_rename = df_precipitacion_completo_knn.copy()
#df_precipitacion_rename['colombia'] = df_precipitacion_rename.sum(axis=1, numeric_only=True)
#df_precipitacion_rename["departamentos_represa"] = df_precipitacion_rename[departamentos].sum(axis=1)
print(df_precipitacion_rename.shape)
#df_precipitacion_rename.drop(columns=departamentos,inplace=True)
#print(df_precipitacion_rename.shape)
#print(df_precipitacion_rename[departamentos].shape[1] == len(departamentos))
# Agregar el prefijo 'precipitacion_' a los nombres de las columnas
df_precipitacion_rename.columns = ['precipitacion_' + col if col not in ['date'] else col for col in df_precipitacion_rename.columns]

(1003, 35)


In [42]:
df_precipitacion_rename.head()

,date,precipitacion_unidadmedida,precipitacion_amazonas,precipitacion_antioquia,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_boyaca,precipitacion_caldas,...,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_santander,precipitacion_sucre,precipitacion_tolima,precipitacion_valle del cauca,precipitacion_vaupes,precipitacion_vichada
0,2021-01-01,mm,49.110000,227.376000,7.800000,11.230000,0.110000,10.500000,57.70000,28.30000,...,4.480000,0.100000,2.300000,0.300000,19.200000,3.400000,107.100000,18.200000,0.700000,0.480000
1,2021-01-02,mm,23.120000,247.065000,33.300000,7.220000,5.860000,9.100000,740.20000,264.40000,...,0.800000,18.300000,22.026000,9.800000,71.220000,4.000000,43.300000,13.713000,3.700000,7.360000
2,2021-01-03,mm,0.220000,346.492000,37.100000,8.020000,0.000000,5.390000,233.60000,95.10000,...,3.200000,42.300000,68.308000,6.400000,9.460000,3.500000,37.300000,141.056000,0.200000,8.700000
3,2021-01-04,mm,0.010000,78.964000,1.700000,8.240000,0.000000,0.000000,23.20000,53.80000,...,4.800000,41.700000,29.418000,1.000000,14.710000,0.600000,21.400000,41.300000,0.000000,15.480000
4,2021-01-05,mm,9.266018,97.365291,18.861241,33.973184,9.864575,45.772519,115.17397,89.92912,...,13.992165,28.172589,40.044579,7.959264,55.149938,23.524713,61.121242,68.206322,4.177108,7.151163


In [43]:
df_precipitacion_rename.to_csv("processed_tables/precipitacion.csv", index=False)
df_precipitacion_rename.head()

,date,precipitacion_unidadmedida,precipitacion_amazonas,precipitacion_antioquia,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_boyaca,precipitacion_caldas,...,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_santander,precipitacion_sucre,precipitacion_tolima,precipitacion_valle del cauca,precipitacion_vaupes,precipitacion_vichada
0,2021-01-01,mm,49.110000,227.376000,7.800000,11.230000,0.110000,10.500000,57.70000,28.30000,...,4.480000,0.100000,2.300000,0.300000,19.200000,3.400000,107.100000,18.200000,0.700000,0.480000
1,2021-01-02,mm,23.120000,247.065000,33.300000,7.220000,5.860000,9.100000,740.20000,264.40000,...,0.800000,18.300000,22.026000,9.800000,71.220000,4.000000,43.300000,13.713000,3.700000,7.360000
2,2021-01-03,mm,0.220000,346.492000,37.100000,8.020000,0.000000,5.390000,233.60000,95.10000,...,3.200000,42.300000,68.308000,6.400000,9.460000,3.500000,37.300000,141.056000,0.200000,8.700000
3,2021-01-04,mm,0.010000,78.964000,1.700000,8.240000,0.000000,0.000000,23.20000,53.80000,...,4.800000,41.700000,29.418000,1.000000,14.710000,0.600000,21.400000,41.300000,0.000000,15.480000
4,2021-01-05,mm,9.266018,97.365291,18.861241,33.973184,9.864575,45.772519,115.17397,89.92912,...,13.992165,28.172589,40.044579,7.959264,55.149938,23.524713,61.121242,68.206322,4.177108,7.151163


se identifica que hay una ausencia de data constante en diciembre y enero, algunos departamentos con un vacio mayor, por lo que se completa la información con KNNImputer que muestra mayor logíca en las graficas.
